### Importing Required Packages and checking version of TensorFlow library

In [1]:
# Importing Required Packages
import numpy as np
import sys
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import tensorflow as tf
plt.style.use('dark_background')
%matplotlib inline
tf.__version__

'2.15.0'

In [2]:
# Setting precision of printing
np.set_printoptions(precision=2)

### Generating Artificial Data i.e. Data matrix (X) and output class labels (y)
* One-hot encoding of  the output class labels y  is done using tensorflow library

In [3]:
# Consider 10 samples, 5 features per sample and 3 output classes
num_samples = 10
num_features = 5
num_labels = 3

# Data matrix (each column = single sample)
X = np.random.choice(np.arange(3, 10), size = (num_features, num_samples), replace = True)

# Output Class labels
y = np.random.choice([0, 1, 2], size = num_samples, replace = True)

print('X = ')
print(X)
print('------')
print('y = ')
print(y)
print('------')

# One-hot encode class labels
y = tf.keras.utils.to_categorical(y).T
print(y)
print('------')
print('Shape of Data Matrix:', X.shape)

X = 
[[6 8 4 4 6 3 9 5 6 5]
 [4 4 7 6 7 4 6 3 5 8]
 [4 5 5 4 7 7 6 9 5 7]
 [5 6 3 9 6 5 5 7 8 9]
 [5 6 8 9 5 6 9 4 8 8]]
------
y = 
[1 2 0 0 0 0 2 2 1 0]
------
[[0. 0. 1. 1. 1. 1. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 1. 1. 0. 0.]]
------
Shape of Data Matrix: (5, 10)


### Generic Layer Class with Forward and Backward method

In [4]:
class Layer:
  def __init__(self): # This is a constructor
    self.input = None # Attribute 1 for the layer
    self.output = None # Attribute 2 for the layer

  def forward(self, input):
    pass

  def backward(self, output_gradient, learning_rate):
    pass

### Define the Dense Layer class with Forward and Backward Method

In [13]:
class Dense(Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.empty((output_size, input_size+1), dtype = np.float64) # Bias Trick
        self.weights[:, :-1] = 0.01 * np.random.randn(output_size, input_size) # Generating weights part of matrix W
        self.weights[:, -1] = 0.01 # Set all bias values to the same nonzero constant

    def forward(self, input):
        self.input = np.append(input, 1) # Adding Bias feature 1
        self.output = np.dot(self.weights, self.input) # As output of Dense layer is Z = W.X

    def backward(self, output_gradient, learning_rate):
        weights_gradient = np.dot(output_gradient.reshape(-1, 1), self.input.reshape(-1, 1).T)
        input_gradient = np.dot(self.weights.T, output_gradient)
        self.weights = self.weights + learning_rate * (-weights_gradient)
        return input_gradient

### Define the SoftMax Activation Layer with Forward and Backward Method

In [9]:
class Softmax(Layer):
  def forward(self, input):
    self.output = tf.nn.softmax(input).numpy()

  def backward(self, output_gradient, learning_rate = None):
    return(np.dot((np.identity(np.size(self.output))-self.output.T) * self.output, output_gradient))

### Defining the Loss function and its gradient
* Here, consider Categorical Cross Entropy Loss (CCE)

In [12]:
def cce(y, yhat):
  return(-np.sum(y*np.log(yhat)))
  #return(np.dot(-y, np.log(yhat)))

def cce_gradient(y, yhat):
  return(-y/yhat)

# TensorFlow in-built function for categorical crossentropy loss
#cce = tf.keras.losses.CategoricalCrossentropy()

### Forward and backward propagation for the 0th sample

In [15]:
learning_rate = 1e-02 # Define Learning Rate

# Define the neural network architecture

dlayer = Dense(num_features, num_labels) # define the dense layer
#print(dlayer.weights)
softmax = Softmax() # define the softmax activation layer
print('Weights for Dense Layer W is')
print(dlayer.weights) # The last column is always 0.01 as this is the bias
print('---------------------------------------------------------')

# Forward propagation starts here

dlayer.forward(X[:, 0]) # forward prop for the 0th sample
softmax.forward(dlayer.output) # The input of Softmax actn layer is Z, which is the output of dlayer
loss = cce(y[:, 0], softmax.output) # softmax.output gives the predicted probabities vector yhat
print('The Calculated Loss using Forward Propagation Method L is')
print(loss)
print('---------------------------------------------------------')

# Backward propagation starts here

grad = cce_gradient(y[:, 0], softmax.output)
grad = softmax.backward(grad)
grad = dlayer.backward(grad, learning_rate)
print('After Updating the Updated Weights are given as')
print(dlayer.weights)

Weights for Dense Layer W is
[[-0.   -0.01  0.   -0.01  0.01  0.01]
 [ 0.01 -0.02 -0.    0.01 -0.01  0.01]
 [ 0.   -0.   -0.01  0.01  0.01  0.01]]
---------------------------------------------------------
The Calculated Loss using Forward Propagation Method L is
1.1391484979535902
---------------------------------------------------------
After Updating the Updated Weights are given as
[[-0.02 -0.02 -0.01 -0.03 -0.    0.01]
 [ 0.05  0.01  0.03  0.04  0.03  0.02]
 [-0.02 -0.01 -0.02 -0.   -0.    0.01]]


### Forward and Backward Propagation for All Samples
* Consider the batch size 1

In [16]:

# Steps: run over each sample, calculate loss, gradient of loss, and update weights.

learning_rate = 1e-02 # Learning Rate

# Define the neural network architecture
dlayer = Dense(num_features, num_labels) # define the dense layer

#print(dlayer.weights)

softmax = Softmax() # define the softmax activation layer

loss = 0.0

for i in range(X.shape[1]):
  # Forward propagation starts here
  dlayer.forward(X[:, i]) # forward prop for the 0th sample
  softmax.forward(dlayer.output)
  loss = cce(y[:, i], softmax.output)

  # Backward propagation starts here
  grad = cce_gradient(y[:, i], softmax.output)
  grad = softmax.backward(grad)
  grad = dlayer.backward(grad, learning_rate)

  print('Sample %d, loss = %f'%(i, loss))

Sample 0, loss = 1.207796
Sample 1, loss = 1.713001
Sample 2, loss = 2.086332
Sample 3, loss = 1.012696
Sample 4, loss = 0.225098
Sample 5, loss = 0.067376
Sample 6, loss = 5.987081
Sample 7, loss = 2.967725
Sample 8, loss = 6.293487
Sample 9, loss = 0.130480
